# Plot import

In [1]:
from datetime import datetime
from typing import  Dict
from vnpy.trader.constant import Exchange, Interval
from vnpy.trader.database import database_manager
from vnpy.app.cta_strategy import ArrayManager
from vnpy.chart import ChartWidget, VolumeItem, CandleItem
import pyqtgraph as pg
from vnpy.trader.ui import create_qapp, QtCore, QtGui
from vnpy.trader.object import BarData
from vnpy.chart.manager import BarManager
from typing import List, Dict, Tuple

In [2]:
app = create_qapp()

# Backtesting Import

In [3]:
from datetime import datetime
from datetime import timedelta
from importlib import reload
from collections import defaultdict

import vnpy.app.portfolio_strategy
reload(vnpy.app.portfolio_strategy)

from vnpy.app.portfolio_strategy import BacktestingEngine
from vnpy.trader.constant import Interval
from vnpy.trader.utility import BarGenerator, ArrayManager

# Strategy

In [4]:
import re

from typing import List, Dict
from datetime import datetime

import numpy as np

from vnpy.app.portfolio_strategy import StrategyTemplate, StrategyEngine
from vnpy.trader.utility import BarGenerator
from vnpy.trader.object import TickData, BarData


In [124]:
class BasisBollStrategy(StrategyTemplate):
    """"""

    author = "chendi"

    # params
    min_basis = 0
    underlying = "000905.SSE"
    basis_return_only = False

    # vars
    underlying_bars = []
    basis_bars = []
    basis_am = ArrayManager(2000)
    target_pos = {}
    over_up = False
    below_down = False
    
    parameters = [
        "min_basis",
        "underlying",
        "basis_return_only",
    ]
    variables = [
    ]

    def __init__(
        self,
        strategy_engine: StrategyEngine,
        strategy_name: str,
        vt_symbols: List[str],
        setting: dict
    ):
        """"""
        super().__init__(strategy_engine, strategy_name, vt_symbols, setting)

        self.targets: Dict[str, int] = {}

    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")

        self.load_bars(120)

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

    def get_expired_date(self, code):
        if code.endswith(".CFFEX"):
            expire_month_str = re.match("IC(\d+).CFFEX", code).groups()[0]
            expire_month_date = datetime.strptime(expire_month_str + "15", "%y%m%d")
            expire_month_date = expire_month_date.replace(day=(15 + (4 - expire_month_date.weekday()) % 7))
            return expire_month_date
        else:
            return None
        
    def on_bars(self, bars: Dict[str, BarData]):
        """""" 
        self.cancel_all()
        
        current_date = list(bars.values())[0].datetime.replace(tzinfo=None)
        print(current_date, self.pos)

        code_list = sorted(filter(lambda x: x.endswith(".CFFEX"), bars.keys()))
        
        # Calculate basis
        basis_map = {}
        for code in code_list:
            basis_point = bars[self.underlying].close_price - bars[code].close_price
            days_to_expire = (self.get_expired_date(code) - current_date).days
            # Daily basis percentage
            if days_to_expire == 0:
                adj_basis_point = -100
            else:
                adj_basis_point = basis_point * 100 / bars[self.underlying].close_price / days_to_expire * 365
            basis_map[code] = adj_basis_point

        #print(basis_map)
        
        # Update basis bar
        adj_basis_list = [ basis_map[code] for code in code_list[1:]]
        avg_adj_basis = sum(adj_basis_list) / len(adj_basis_list)
        #print(avg_adj_basis)
        underlying_bar = bars[self.underlying]
        self.underlying_bars.append(underlying_bar)

        basis_bar = BarData(gateway_name='backtesting', 
                          symbol=underlying_bar.symbol + "basis",  
                          exchange=underlying_bar.exchange, 
                          datetime=underlying_bar.datetime,
                          interval=underlying_bar.interval,
                          volume=underlying_bar.volume,
                          open_price=avg_adj_basis,
                          high_price=avg_adj_basis,
                          low_price=avg_adj_basis,
                          close_price=avg_adj_basis
                         )            
        self.basis_am.update_bar(basis_bar)
        
        if self.basis_am.count < 120:
            print("Not init, skip trading: {}".format(self.basis_am.count))
            return

        # Check basis boll
        boll_up, boll_down = self.basis_am.boll(120, 2)
        
        # basis cross from up to down, basis get into down trend
        if self.over_up:
            # Buy far month contract, sell front month contract
            self.target_pos = {
                code_list[-1] : 1,
                code_list[1] : -1,
            }
            print("{} Over up, curr basis {} target: {}".format(current_date, avg_adj_basis, self.target_pos))
            
        if self.below_down:
            self.target_pos = {
                code_list[-1] : -1,
                code_list[1] : 1,
            }
            print("{} Below down, curr basis {} target: {}".format(current_date, avg_adj_basis, self.target_pos))


        sma_120 = self.basis_am.sma(120, array=True)
        if (self.basis_am.close[-1] - sma_120[-1]) * (self.basis_am.close[-2] - sma_120[-2]) < 0:
            # Cross the middle line, reset target pos
            self.target_pos = {}
            print("Clear target pos")
            
        self.over_up = avg_adj_basis > boll_up
        self.below_down = avg_adj_basis < boll_down       
        
        # Roll position is close to expiration date
        for symbol, pos in self.pos.items():
            if pos == 0:
                continue
            expire_date = self.get_expired_date(symbol)
            if (expire_date - current_date).days > 2:
                continue
            
            if pos > 0:
                self.target_pos = {
                    code_list[-1] : -1,
                    code_list[1] : 1,
                }
            else:
                self.target_pos = {
                    code_list[-1] : 1,
                    code_list[1] : -1,
                }
            
            print("{} Roll {} {} -> {}".format(current_date, pos, symbol, self.target_pos))

        # Execute trade
        symbol_set = set(list(self.target_pos.keys()) + list(self.pos.keys()))
        for symbol in symbol_set:
            position_change = self.target_pos.get(symbol, 0) - self.pos.get(symbol, 0)
            if position_change > 0:
                self.buy(symbol, bars[symbol].close_price * 1.2, position_change)
            elif position_change < 0:
                self.sell(symbol, bars[symbol].close_price * 0.8, 0 - position_change)

        self.put_event()

In [125]:
#%%

def get_ic_symbol_between(start, end):
    symbols = []
    while start < (end + timedelta(days=360)):
        start = (start + timedelta(days=32)).replace(day=1)
        date_str = start.strftime("%y%m")
        symbols.append("IC" + date_str + ".CFFEX")
    return symbols

start = datetime(2017,1,1)
end = datetime.today()

symbols = ["000905.SSE"] + get_ic_symbol_between(start, end)
rates = defaultdict(lambda:0)
slippages = defaultdict(lambda:0)
sizes = defaultdict(lambda:300)
priceticks = defaultdict(lambda:0.1)

    
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbols=symbols,
    interval=Interval.DAILY,
    start=start,
    end=end,
    rates=rates,
    slippages=slippages,
    sizes=sizes,
    priceticks=priceticks,
    capital=1_000_000,
)

setting = {
    "min_basis": 0,
    "underlying": '000905.SSE',
    "basis_return_only": True
}
engine.add_strategy(BasisBollStrategy, setting)

In [129]:
engine.trades

{'BACKTESTING.1': TradeData(gateway_name='BACKTESTING', symbol='IC1803', exchange=<Exchange.CFFEX: 'CFFEX'>, orderid='1', tradeid='1', direction=<Direction.SHORT: '空'>, offset=<Offset.CLOSE: '平'>, price=6466.4, volume=1, datetime=datetime.datetime(2017, 9, 20, 0, 0, tzinfo=<StaticTzInfo 'Etc/GMT'>)),
 'BACKTESTING.2': TradeData(gateway_name='BACKTESTING', symbol='IC1711', exchange=<Exchange.CFFEX: 'CFFEX'>, orderid='2', tradeid='2', direction=<Direction.LONG: '多'>, offset=<Offset.OPEN: '开'>, price=6575.0, volume=1, datetime=datetime.datetime(2017, 9, 20, 0, 0, tzinfo=<StaticTzInfo 'Etc/GMT'>)),
 'BACKTESTING.3': TradeData(gateway_name='BACKTESTING', symbol='IC1803', exchange=<Exchange.CFFEX: 'CFFEX'>, orderid='3', tradeid='3', direction=<Direction.LONG: '多'>, offset=<Offset.OPEN: '开'>, price=6419.8, volume=1, datetime=datetime.datetime(2017, 10, 19, 0, 0, tzinfo=<StaticTzInfo 'Etc/GMT'>)),
 'BACKTESTING.4': TradeData(gateway_name='BACKTESTING', symbol='IC1711', exchange=<Exchange.CFFEX

In [127]:
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()

2021-06-27 12:49:19.823163	开始加载历史数据
2021-06-27 12:49:19.827163	000905.SSE加载进度： [2%]
2021-06-27 12:49:19.829162	000905.SSE加载进度： [4%]
2021-06-27 12:49:19.832162	000905.SSE加载进度： [5%]
2021-06-27 12:49:19.836163	000905.SSE加载进度： [7%]
2021-06-27 12:49:19.839161	000905.SSE加载进度： [9%]
2021-06-27 12:49:19.843163	000905.SSE加载进度：# [11%]
2021-06-27 12:49:19.846163	000905.SSE加载进度：# [13%]
2021-06-27 12:49:19.849163	000905.SSE加载进度：# [15%]
2021-06-27 12:49:19.853163	000905.SSE加载进度：# [16%]
2021-06-27 12:49:19.856172	000905.SSE加载进度：# [18%]
2021-06-27 12:49:19.859163	000905.SSE加载进度：## [20%]
2021-06-27 12:49:19.863163	000905.SSE加载进度：## [22%]
2021-06-27 12:49:19.867164	000905.SSE加载进度：## [24%]
2021-06-27 12:49:19.869169	000905.SSE加载进度：## [26%]
2021-06-27 12:49:19.872164	000905.SSE加载进度：## [27%]
2021-06-27 12:49:19.875163	000905.SSE加载进度：## [29%]
2021-06-27 12:49:19.878164	000905.SSE加载进度：### [31%]
2021-06-27 12:49:19.882164	000905.SSE加载进度：### [33%]
2021-06-27 12:49:19.885672	000905.SSE加载进度：### [35%]
2021-06-27 1

2021-06-27 12:49:20.186691	IC1705.CFFEX加载进度：#### [40%]
2021-06-27 12:49:20.188690	IC1705.CFFEX加载进度：#### [42%]
2021-06-27 12:49:20.190691	IC1705.CFFEX加载进度：#### [44%]
2021-06-27 12:49:20.191691	IC1705.CFFEX加载进度：#### [46%]
2021-06-27 12:49:20.193691	IC1705.CFFEX加载进度：#### [48%]
2021-06-27 12:49:20.194692	IC1705.CFFEX加载进度：#### [49%]
2021-06-27 12:49:20.195692	IC1705.CFFEX加载进度：##### [51%]
2021-06-27 12:49:20.197691	IC1705.CFFEX加载进度：##### [53%]
2021-06-27 12:49:20.198691	IC1705.CFFEX加载进度：##### [55%]
2021-06-27 12:49:20.200690	IC1705.CFFEX加载进度：##### [57%]
2021-06-27 12:49:20.201691	IC1705.CFFEX加载进度：##### [59%]
2021-06-27 12:49:20.202691	IC1705.CFFEX加载进度：###### [60%]
2021-06-27 12:49:20.204692	IC1705.CFFEX加载进度：###### [62%]
2021-06-27 12:49:20.205691	IC1705.CFFEX加载进度：###### [64%]
2021-06-27 12:49:20.207691	IC1705.CFFEX加载进度：###### [66%]
2021-06-27 12:49:20.208691	IC1705.CFFEX加载进度：###### [68%]
2021-06-27 12:49:20.209691	IC1705.CFFEX加载进度：###### [70%]
2021-06-27 12:49:20.211691	IC1705.CFFEX加载进度：####

2021-06-27 12:49:20.561698	IC1710.CFFEX加载进度：######## [86%]
2021-06-27 12:49:20.562696	IC1710.CFFEX加载进度：######## [88%]
2021-06-27 12:49:20.563698	IC1710.CFFEX加载进度：######## [90%]
2021-06-27 12:49:20.565700	IC1710.CFFEX加载进度：######### [92%]
2021-06-27 12:49:20.566698	IC1710.CFFEX加载进度：######### [93%]
2021-06-27 12:49:20.568699	IC1710.CFFEX加载进度：######### [95%]
2021-06-27 12:49:20.569698	IC1710.CFFEX加载进度：######### [97%]
2021-06-27 12:49:20.569698	IC1710.CFFEX历史数据加载完成，数据量：40
2021-06-27 12:49:20.571698	IC1711.CFFEX加载进度： [2%]
2021-06-27 12:49:20.572698	IC1711.CFFEX加载进度： [4%]
2021-06-27 12:49:20.573698	IC1711.CFFEX加载进度： [5%]
2021-06-27 12:49:20.575698	IC1711.CFFEX加载进度： [7%]
2021-06-27 12:49:20.576698	IC1711.CFFEX加载进度： [9%]
2021-06-27 12:49:20.577699	IC1711.CFFEX加载进度：# [11%]
2021-06-27 12:49:20.579699	IC1711.CFFEX加载进度：# [13%]
2021-06-27 12:49:20.580699	IC1711.CFFEX加载进度：# [15%]
2021-06-27 12:49:20.582698	IC1711.CFFEX加载进度：# [16%]
2021-06-27 12:49:20.586698	IC1711.CFFEX加载进度：# [18%]
2021-06-27 12:49:2

2021-06-27 12:49:20.936704	IC1804.CFFEX加载进度：## [20%]
2021-06-27 12:49:20.937704	IC1804.CFFEX加载进度：## [22%]
2021-06-27 12:49:20.938704	IC1804.CFFEX加载进度：## [24%]
2021-06-27 12:49:20.941706	IC1804.CFFEX加载进度：## [26%]
2021-06-27 12:49:20.943706	IC1804.CFFEX加载进度：## [27%]
2021-06-27 12:49:20.946705	IC1804.CFFEX加载进度：## [29%]
2021-06-27 12:49:20.947706	IC1804.CFFEX加载进度：### [31%]
2021-06-27 12:49:20.949706	IC1804.CFFEX加载进度：### [33%]
2021-06-27 12:49:20.950706	IC1804.CFFEX加载进度：### [35%]
2021-06-27 12:49:20.951705	IC1804.CFFEX加载进度：### [37%]
2021-06-27 12:49:20.953707	IC1804.CFFEX加载进度：### [38%]
2021-06-27 12:49:20.956704	IC1804.CFFEX加载进度：#### [40%]
2021-06-27 12:49:20.958705	IC1804.CFFEX加载进度：#### [42%]
2021-06-27 12:49:20.959706	IC1804.CFFEX加载进度：#### [44%]
2021-06-27 12:49:20.961706	IC1804.CFFEX加载进度：#### [46%]
2021-06-27 12:49:20.962706	IC1804.CFFEX加载进度：#### [48%]
2021-06-27 12:49:20.964706	IC1804.CFFEX加载进度：#### [49%]
2021-06-27 12:49:20.965706	IC1804.CFFEX加载进度：##### [51%]
2021-06-27 12:49:20.966705

2021-06-27 12:49:21.311712	IC1809.CFFEX加载进度：####### [71%]
2021-06-27 12:49:21.312712	IC1809.CFFEX加载进度：####### [73%]
2021-06-27 12:49:21.313711	IC1809.CFFEX加载进度：####### [75%]
2021-06-27 12:49:21.315713	IC1809.CFFEX加载进度：####### [77%]
2021-06-27 12:49:21.316712	IC1809.CFFEX加载进度：####### [79%]
2021-06-27 12:49:21.318713	IC1809.CFFEX加载进度：######## [81%]
2021-06-27 12:49:21.319713	IC1809.CFFEX加载进度：######## [82%]
2021-06-27 12:49:21.320713	IC1809.CFFEX加载进度：######## [84%]
2021-06-27 12:49:21.322713	IC1809.CFFEX加载进度：######## [86%]
2021-06-27 12:49:21.323713	IC1809.CFFEX加载进度：######## [88%]
2021-06-27 12:49:21.324714	IC1809.CFFEX加载进度：######## [90%]
2021-06-27 12:49:21.326714	IC1809.CFFEX加载进度：######### [92%]
2021-06-27 12:49:21.327715	IC1809.CFFEX加载进度：######### [93%]
2021-06-27 12:49:21.328714	IC1809.CFFEX加载进度：######### [95%]
2021-06-27 12:49:21.330716	IC1809.CFFEX加载进度：######### [97%]
2021-06-27 12:49:21.330716	IC1809.CFFEX历史数据加载完成，数据量：165
2021-06-27 12:49:21.332714	IC1810.CFFEX加载进度： [2%]
2021-06-27

...

2019-08-07 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 1, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': -1})
2019-08-08 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 1, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': -1})
2019-08-09 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 

2020-04-16 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': 0, 'IC2001.CFFEX': 0, 'IC2002.CFFEX': 0, 'IC2004.CFFEX': 0, 'IC2009.CFFEX': 1, 'IC2005.CFFEX': -1})
2020-04-17 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': 0, 'IC2001.CFFEX': 0, 'IC2002.CFFEX': 0, 'IC2004.CFFEX': 0, 'IC2009.CFFEX': 1, 'IC2005.C

2020-05-18 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': -1, 'IC2001.CFFEX': 0, 'IC2002.CFFEX': 0, 'IC2004.CFFEX': 0, 'IC2009.CFFEX': 0, 'IC2005.CFFEX': 0, 'IC2012.CFFEX': 1})
2020-05-19 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': -1, 'IC2001.CFFEX': 0, 'IC2002.CFFEX': 0, 'IC2004.CFFEX': 0, 'IC2009.

2020-12-23 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': 0, 'IC2001.CFFEX': 0, 'IC2002.CFFEX': 0, 'IC2004.CFFEX': 0, 'IC2009.CFFEX': 0, 'IC2005.CFFEX': 0, 'IC2012.CFFEX': 0, 'IC2007.CFFEX': 0, 'IC2008.CFFEX': 0})
2020-12-24 00:00:00 defaultdict(<class 'int'>, {'IC1803.CFFEX': 0, 'IC1711.CFFEX': 0, 'IC1802.CFFEX': 0, 'IC1806.CFFEX': 0, 'IC1809.CFFEX': 0, 'IC1805.CFFEX': 0, 'IC1808.CFFEX': 0, 'IC1812.CFFEX': 0, 'IC1811.CFFEX': 0, 'IC1903.CFFEX': 0, 'IC1906.CFFEX': 0, 'IC1909.CFFEX': 0, 'IC1904.CFFEX': 0, 'IC1905.CFFEX': 0, 'IC1912.CFFEX': 0, 'IC1907.CFFEX': 0, 'IC1908.CFFEX': 0, 'IC2003.CFFEX': 0, 'IC2006.CFFEX': 0, 'IC2001.CFFEX': 0, 'IC2002.CF

2021-06-27 12:49:25.774803	逐日盯市盈亏计算完成


In [128]:
engine.calculate_statistics()
engine.show_chart()

2021-06-27 12:49:25.795803	开始计算策略统计指标
2021-06-27 12:49:25.807803	------------------------------
2021-06-27 12:49:25.807803	首个交易日：	2017-01-03
2021-06-27 12:49:25.808802	最后交易日：	2021-06-25
2021-06-27 12:49:25.808802	总交易日：	1089
2021-06-27 12:49:25.808802	盈利交易日：	163
2021-06-27 12:49:25.808802	亏损交易日：	191
2021-06-27 12:49:25.808802	起始资金：	1,000,000.00
2021-06-27 12:49:25.809802	结束资金：	892,120.00
2021-06-27 12:49:25.809802	总收益率：	-10.79%
2021-06-27 12:49:25.809802	年化收益：	-2.38%
2021-06-27 12:49:25.809802	最大回撤: 	-187,920.00
2021-06-27 12:49:25.810801	百分比最大回撤: -18.59%
2021-06-27 12:49:25.810801	最长回撤天数: 	862
2021-06-27 12:49:25.810801	总盈亏：	-107,880.00
2021-06-27 12:49:25.810801	总手续费：	0.00
2021-06-27 12:49:25.810801	总滑点：	0.00
2021-06-27 12:49:25.811801	总成交金额：	129,058,080.00
2021-06-27 12:49:25.811801	总成交笔数：	80
2021-06-27 12:49:25.811801	日均盈亏：	-99.06
2021-06-27 12:49:25.811801	日均手续费：	0.00
2021-06-27 12:49:25.812802	日均滑点：	0.00
2021-06-27 12:49:25.812802	日均成交金额：	118,510.63
2021-06-27 12:49:25.812802	日均成交

# Plot candle graph

In [9]:
class BaseIndicatorLine(CandleItem):
    """自定义指标显示"""
    bar_attr_name="down_line"
    rgb_color = (100, 100, 255)
    
    def __init__(self, manager: BarManager):
        """"""
        super().__init__(manager)

        self.blue_pen: QtGui.QPen = pg.mkPen(color=self.rgb_color, width=2)
        self.indicator_data: Dict[int, float] = {}

    def get_indicator_value(self, ix: int) -> float:
        """"""
        if not self.indicator_data:
            bars = self._manager.get_all_bars()
            indicator_array = [getattr(bar, self.bar_attr_name) for bar in bars]

            for n, value in enumerate(indicator_array):
                self.indicator_data[n] = value

        return self.indicator_data.get(ix, 0)


    def _draw_bar_picture(self, ix: int, bar: BarData) -> QtGui.QPicture:
        """"""
        indicator_value = self.get_indicator_value(ix)
        last_indicator_value = self.get_indicator_value(ix - 1)

        # Create objects
        picture = QtGui.QPicture()
        painter = QtGui.QPainter(picture)

        # Set painter color
        painter.setPen(self.blue_pen)

        # Draw Line
        start_point = QtCore.QPointF(ix-1, last_indicator_value)
        end_point = QtCore.QPointF(ix, indicator_value)
        painter.drawLine(start_point, end_point)

        # Finish
        painter.end()
        return picture

    def boundingRect(self) -> QtCore.QRectF:
        """"""
        rect = QtCore.QRectF(
            0,
            0,
            len(self._bar_picutures),
            30
        )
        return rect
    
    def get_y_range(self, min_ix: int = None, max_ix: int = None) -> Tuple[float, float]:
        return 0, 30
    
    def get_info_text(self, ix: int) -> str:
        """"""
        value = self.indicator_data.get(ix, "-")
        text = f"{self.bar_attr_name} {value}"
        return text



In [ ]:
# Compute indicator
underlying_bars = engine.strategy.underlying_bars
basis_bars = engine.strategy.basis_bars
assert(len(underlying_bars) == len(basis_bars))
basis_am = ArrayManager(2000)
for ix, bar in enumerate(underlying_bars):
    basis_am.update_bar(basis_bars[ix])
    
    bar.indicator_basis = basis_bars[ix].close_price
    bar.indicator_sma20 = basis_am.sma(20)
    bar.indicator_boll_up, bar.indicator_boll_down = basis_am.boll(20, 1.8)
    
# Start plot
widget = ChartWidget()
widget.add_plot("candle", hide_x_axis=True)
widget.add_plot("indicator", hide_x_axis=True)
widget.add_plot("volume", maximum_height=250)

template_bar = underlying_bars[-1]
color_list = [
    (255, 0, 0),
    (128, 0, 0),
    (255, 255, 0),
    (128, 128, 0),
    (0, 255, 0),
    (0, 128, 0),
    (0, 128, 128),
    (0, 0, 255)
]
curr_color = 0
for attribute_name in dir(template_bar):
    if not attribute_name.startswith("indicator_"):
        continue
    indicator_class = type('SubClass', (BaseIndicatorLine,), {'bar_attr_name': attribute_name, 'rgb_color': color_list[curr_color]})
    widget.add_item(indicator_class, attribute_name, "indicator")
    curr_color += 1

widget.add_item(CandleItem, "candle", "candle")
widget.add_item(VolumeItem, "volume", "volume")


widget.add_cursor()

widget.update_history(underlying_bars)

widget.show()
app.exec_()